In [40]:
import pandas as pd
import autograd.numpy as np
from autograd import grad
from bokeh.plotting import output_notebook, show, figure
output_notebook()

Loading BokehJS ...

## Logistic Regression

Logistic regression is a common method for binary classification problems (problems with two class values). Unlike linear regression which outputs continuous number values, logistic regression transforms its output using the logistic sigmoid function to return a probability value which can then be mapped to two or more discrete classes (only specific values or categories are allowed).

We will be using binary logistic regression and [this dataset](https://github.com/JWarmenhoven/ISLR-python/blob/master/Notebooks/Data/Default.xlsx) to find out how income and account balance affect whether or not a person defaults.

### Analyzing Data

In [41]:
df = pd.read_excel('Default.xlsx')
df.head()

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


In [42]:
pd.read_excel('Default.xlsx')['balance'].values

array([  729.52649521,   817.18040656,  1073.54916401, ...,   845.41198922,
        1569.00905338,   200.92218263])

In [43]:
df['balance'].head()

1     729.526495
2     817.180407
3    1073.549164
4     529.250605
5     785.655883
Name: balance, dtype: float64

In [44]:
type(df['balance'])

pandas.core.series.Series

In [45]:
balance_np = df['balance'].values
income_np = df['income'].values
default_np = df['default'].values

### Graphing Initial Data

In [79]:
color_mapping = {'No': 'green', 'Yes': 'red'}
color_mapped_default_np = np.array([color_mapping[x] for x in default_np])

plot = figure(title='Default Data', x_axis_label='Balance', 
              y_axis_label='Income', plot_width=500, plot_height=500)
plot.scatter(x=balance_np, y=income_np, color=color_mapped_default_np, 
             line_color=None, alpha=0.25)
show(plot)

### Normalizing the Dataset

Using the given dataset causes our model to crash when calculating the loss later, so the data for both the balance and income was rescaled using [feature scaling](http://www.statisticshowto.com/normalized/). This results in all datapoints ranging between 0 and 1.


\\[{x_{new}} = \frac{{{x_i} - \min ({x_i})}}{{\max ({x_i}) - \min ({x_i})}}\\]


\\({x_{new}}\\): normalized dataset

\\({x_i}\\): original dataset

\\(min (x)\\): minimum of original dataset

\\(max (x)\\): maximum of original dataset

In [47]:
balance_normalized_np = balance_np/balance_np.max()
income_normalized_np = income_np/income_np.max()

In [80]:
plot = figure(title='Default information', x_axis_label='Balance', 
              y_axis_label='Income', plot_width=500, plot_height=500)
plot.scatter(x=balance_normalized_np, y=income_normalized_np, 
             color=color_mapped_default_np, line_color=None, alpha=0.25)
show(plot)

In [49]:
logistic_mapping = {'No': 0, 'Yes': 1}
logistic_default_np = np.array([logistic_mapping[x] for x in default_np])

logistic_default_np

array([0, 0, 0, ..., 0, 0, 0])

### Model

The model predicts whether or not the person defaults. There is one weight for each input.

\\[y = {w_0} + {w_1}*{x_1} + {w_2}*{x_2}\\]

\\(w_0\\): slope

\\(x_1\\): the balance

\\(x_2\\): the income

\\(w_1\\): weight for \\(x_1\\)

\\(w_2\\): weight for \\(x_2\\)


The model will try to identify weight values that most reduce our loss function. We will transform the model's output using the sigmoid function to return a probability value between 0 and 1. 

We set the initial weights to 0, which the model will adjust with each iteration.

In [50]:
w = np.array([0., 0., 0.])

def model(w, balance=balance_normalized_np, income=income_normalized_np):
    return sigmoid ((w[0] + w[1] * balance) + (w[2] * income))

### Sigmoid Function

While linear regression outputs continuous number values, logistic regression transforms its output using the sigmoid function. It then returns a probability value which can be mapped to two or more discrete classes. 

\\[S(x) = \frac{1}{{1 + {e^{ - x}}}}\\]

\\(S(x)\\): output between 0 and 1 (probability estimate), model's prediction

\\(x\\): input to the function (algorithm's prediction e.g. mx + b)

\\(e\\): base of natural log


In [51]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

sigmoid(np.linspace(-6,6,20))

array([ 0.00247262,  0.00463986,  0.00869011,  0.01621831,  0.03007034,
        0.05509084,  0.0988092 ,  0.17094461,  0.27941436,  0.42170223,
        0.57829777,  0.72058564,  0.82905539,  0.9011908 ,  0.94490916,
        0.96992966,  0.98378169,  0.99130989,  0.99536014,  0.99752738])

In [52]:
plot = figure(title='Sigmoid figure', plot_width=400, plot_height=300)
plot.line(x=np.linspace(-6,6,100), y=sigmoid(np.linspace(-6,6,100)),
         line_width=4)
show(plot)

### Loss function

The Cross-Entropy function is used for the loss. It can be divided into two separate loss functions for both 1 and 0.

\\[J(\theta ) = \frac{1}{m}\sum\limits_{i = 1}^m {Loss({h_\theta }({x^i}),{y^i})} \\]

<br>

<center>*If y=1*:  \\(Loss({h_\theta }(x),y) =  - \log ({h_\theta }(x))\\)</center>


<center>*If y=0*:  \\(Loss({h_\theta }(x),y) =  - \log (1 - {h_\theta }(x))\\)</center>



In [84]:
def loss(w):
    predictions = model(w)
    label_probabilities = predictions * logistic_default_np + (1 - predictions) * (1 - logistic_default_np)
    return -np.sum(np.log(label_probabilities))

### Gradient Descent

We will take the gradient (the derivative) of the loss function to minimize our loss.

In [92]:
model_grad = grad(loss)

for i in range(10000):
#     print(w)
    w = w - (0.00001 * model_grad(w))

In [86]:
model(w)

array([ 0.00765508,  0.0124907 ,  0.0250631 , ...,  0.01016142,
        0.11141723,  0.0016401 ])

In [93]:
loss(w)

813.88597627734271

In [94]:
w

array([ -8.84474057,  11.31702839,   0.40063472])

## Comparison To Scikit-Learn

In [96]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split